## Instalações

In [ ]:
pip install onesignal-python-api

In [ ]:
import requests
import onesignal
from onesignal.api import default_api
import json
import pandas as pd
from datetime import datetime, date, timedelta

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [ ]:
# Substitua estas variáveis com suas próprias credenciais e URL da API
rest_api_key = 'xxxxxxx'
app_id = 'xxxxx'
api_url = f'https://onesignal.com/api/v1/apps/{app_id}/players'

In [ ]:
# Cabeçalhos necessários para autenticação
headers = {
    'Authorization': f'xxxxxx',
}

In [ ]:
# Faça a solicitação GET para listar os dispositivos
response = requests.get(api_url, headers=headers)

In [ ]:
# Verifique se a solicitação foi bem-sucedida (código de status HTTP 200)
if response.status_code == 200:
    data = response.json()
    print(data)
elif response.status_code == 404:
    print("Página não encontrada")
else:
    print(f'Erro: {response.status_code}, {response.text}')

### csv-endpoint

In [ ]:
header = {
    "Content-Type": "application/json; charset=utf-8",
    "Authorization": "xxxxx"
}

app_id = "xxxxxxx"

url = f"https://onesignal.com/api/v1/players/csv_export?app_id={app_id}&offset=0"

req = requests.post(url, headers=header)

if req.status_code == 200:
    response = json.loads(req.text)
    print(response)
else:
    print(f'Erro: {req.status_code}, {req.text}')


In [ ]:
#Create DataFrame
df = pd.DataFrame(response, index=[0])
print(df)

In [ ]:
##since response is a AWS s3 link, this section is pulling only the last row of the column csv_file_url which contains the URL
df=df['csv_file_url'].iloc[-1]

In [ ]:
## to get data from the AWS S3 link, we have to unzip it
data = pd.read_csv(df,compression='gzip')

##Save unzipped format to a dataframe df
df = pd.DataFrame(data)


#Pull only specific metrics
df=df.loc[:,['created_at','invalid_identifier']]
print(df)

#Create brandname to differentiate data from one appid to another. Here, I am creating a column called "name" and assigning it whatever name I'll decide on
df['name'] = '{insert-name-you`d-want}'


#remove timestamp from date
df['created_at'] = pd.to_datetime(df['created_at']).dt.date
df

In [ ]:
df. info()

In [ ]:
# Especifique o caminho do arquivo CSV onde você deseja salvar o DataFrame
caminho_arquivo_csv = 'onesignal_endpoint.csv'

# Use o método to_csv() para salvar o DataFrame como um arquivo CSV
df.to_csv(caminho_arquivo_csv, index=False)

## **view-notifications-endpointV2**

In [ ]:
header = {
    "Content-Type": "application/json; charset=utf-8",
    "Authorization": "xxxxxx"
}

app_id = "xxxxxx"

url = f"https://onesignal.com/api/v1/notifications?app_id={app_id}"

req = requests.get(url, headers=header)

if req.status_code == 200:
    response = json.loads(req.text)
    print(response)
else:
    print(f'Erro: {req.status_code}, {req.text}')

In [ ]:
df=pd.json_normalize(response)
df

In [ ]:
df=pd.json_normalize(response, record_path =['notifications'])
df

In [ ]:
df.columns.tolist()

In [ ]:
colunas_a_excluir = [coluna for coluna in df.columns if all(df[coluna].isnull() | (df[coluna] == 0.0))]
colunas_a_excluir

In [ ]:
df = df.drop(columns=colunas_a_excluir)

In [ ]:
df2 = pd.json_normalize(response, 'notifications', ['completed_at', 'converted', 'received','successful'],
                    record_prefix='notifications_', errors='ignore')
df2

In [ ]:
colunas_a_excluir2 = [coluna for coluna in df2.columns if all(df2[coluna].isnull() | (df2[coluna] == 0.0))]
colunas_a_excluir2

In [ ]:
df2 = df2.drop(columns=colunas_a_excluir2)
df2

In [ ]:
df2.info()

In [ ]:
# Converter 'notifications_completed_at' para datetime
df2['notifications_completed_at'] = pd.to_datetime(df2['notifications_completed_at'])

# Converter 'notifications_completed_at' para o formato "DD/MM/AA"
df2['notifications_completed_at'] = df2['notifications_completed_at'].dt.strftime('%d/%m/%y')

# Repita o mesmo processo para 'notifications_queued_at'
df2['notifications_queued_at'] = pd.to_datetime(df2['notifications_queued_at'])
df2['notifications_queued_at'] = df2['notifications_queued_at'].dt.strftime('%d/%m/%y')

# E também para 'notifications_send_after'
df2['notifications_send_after'] = pd.to_datetime(df2['notifications_send_after'])
df2['notifications_send_after'] = df2['notifications_send_after'].dt.strftime('%d/%m/%y')

df2

In [ ]:
# Especifique o caminho do arquivo CSV onde você deseja salvar o DataFrame
caminho_arquivo_csv = 'onesignal_viewnotificationsendpoint.csv'

# Use o método to_csv() para salvar o DataFrame como um arquivo CSV
df2.to_csv(caminho_arquivo_csv, index=False)

In [ ]:
tarefas_setembro_df2 = df2.dropna(subset=['notifications_completed_at'])
tarefas_setembro_df2 = tarefas_setembro_df2[tarefas_setembro_df2['notifications_completed_at'].str.contains("/09/")]

tarefas_setembro_df2

In [ ]:
soma_notifications_successful = tarefas_setembro_df2['notifications_successful'].sum()
print(f"A soma de notificações bem sucedidas é: {soma_notifications_successful}")